<h1 style="text-align:center; color:#207a38;">ROSES_2020 Unit 11 —- Gridding-Inversion-Tomography</h1>
<br>
<br>
<div style="text-align:center; font-size:16px">
    <b><a href="http://geophysics.earth.northwestern.edu/seismology/suzan/">Suzan van der Lee</a></b><br>
    <br>
    Department of Earth and Planetary Sciences<br>
    Integrated Data-Driven Discovery in Earth and Astro-physical Sciences (IDEAS)<br> 
    Northwestern Institute on Complex Systems<br>
    Northwestern University<br>
    Evanston, Illinois, USA<br>
    <br>
    1 September 2020
</div>

- - -

# Mapping the depth to the Moho beneath the USA (48 states)

In geophysics, the bottom of the crust is referred to as the **Mohorovičić Discontinuity**, after the Croatian seismologist who first discovered it. The disconintuity is nick-named "Moho". 

In this Problem Set you will use estimates of Moho depth beneath seismic stations to infer Moho depth elsewhere, where there are no stations, or where we have independent estimates of Moho depth that were not included in the inference. In ML lingo, we could say that we are using observations of Moho depth at a finite set of inputs, called the training set, in order to predict Moho depth anywhere, in particular for test inputs, where we can compare the predictions with observations of Moho depth not included in the training set. 

You have already learned in unit 10 that you can accomplish this with the method of kriging. Likewise pyGMT, which was covered in unit 8, provides a range of methods for spatial averaging.

Here we focus on the method of **gridding**, which requires **inversion** of a forward problem in a similar manner as used in seismic **tomography**.

*Note:* 
The code provided in this notebook was developed to minimize dependencies on imports and to teach the concepts underlying gridding, inversion, and tomography and therefore, they are far from computationally efficient. Your actual research will benefit from your using an established Python module or other software package that is optimized, robust, and open source. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import csv
radius = 6371.  # km (Earth)

Run the above imports and the distance function below. It's incredible that the `scipy.spatial.distance.cdist` function can calculate nearly two dozen different types of distances and "distances", but the actual great-circle segment distance between two points on a sphere is not among them. Earth science to the rescue: the function below, `gcdist()`, will provide just that. 

In [ ]:
def gcdist(xa,xb):
    """
    Function to compute distance on a sphere (Earth):
    IN: xa and xb need to be n x 2 matrices representing n points on a map, that is a set of n row vectors. 
    Within a row, longitude is first, latitude second, both in degrees.
    OUT: a len(xa) by len(xb) array of distances, in km. 
    """  
    xar = np.radians(xa); xbr = np.radians(xb)
    D = []
    for a in xar:
        term1 = np.sin(a[1])*np.sin(xbr[:,1])
        term2 = np.cos(a[1])*np.cos(xbr[:,1])*np.cos(a[0]-xbr[:,0])
        tsum = term1 + term2
        tsum[tsum < -1.] = [-1.]; tsum[tsum>1.0] = [1.0]
        delta = np.arccos(tsum)
        D.append(delta*radius)
    return np.array(D)
    

The **EARS** project provides estimates/observations/measurements of Moho depth at the locations of seismoc stations in the USA. These data are provided through https://ds.iris.edu/ds/products/ears/. The observations have been extracted from **USArray** and other seismograms in an automated manner. The quality of these estimates of Moho depth is reflected by the provided formal noise estimates, which we will use in this problem set. It is possible that these noise estimates are underestimated.

Check out the maps on the EARS web page, in particular this one: http://ds.iris.edu/files/products/ears/map/thickness.jpg. For your convenience I have downloaded and include a CSV file of these data points with this Problem Set, called `USA_average.csv`.


### Reading and visualizing the data

First we need to read the data from the provided file `USA_average.csv`. Alternatively you may download the data directly from the web site referenced above. We will split the data set into a **training set and a testing set**. For the training set we are going to use Moho estimates from all stations that are not part of the Transportable Array. In some, rather inaccurate way this reflects the seismic station distribution before EarthScope, or at least we will pretend it is. In reality, this includes many stations from EarthScope's Flexible Array experiments conducted by small groups of researchers. 

We will use the Moho estimates made for TA stations as the testing set, i.e. the data set with which we compare predictions made using our various gridding methods (without and with inversion). 

In [ ]:
# Open file with Moho measurements
ears = csv.reader(open('USA_average.csv', 'r'))

# create empty lists for training and testing sets:
xlist = []; ylist = []; elist = []
xlist2 = []; ylist2 = []; elist2 = []

# read in actual data (Moho measurements) and estimated uncertainties
for row in ears:
    if not row[0].startswith('#'):
        (net, sta) = (row[0],row[1])
        if net != "TA":
            xlist.append(np.array([float(row[3]),float(row[2])]))                   # lon and lat (in degrees)
            ylist.append(float(row[5].split()[0])-0.001*float(row[4].split()[0]))   # Moho depth = crustal thickness - topography (in km)
            elist.append(float(row[6].split()[0]))                                  # Error estimate (in km)
        else:
            xlist2.append(np.array([float(row[3]),float(row[2])])) 
            ylist2.append(float(row[5].split()[0])-0.001*float(row[4].split()[0]))
            elist2.append(float(row[6].split()[0]))

#convert lists to arrays for computational reasons           
xobs = np.array(xlist); yobs = np.array(ylist); eobs = np.array(elist)
xtest = np.array(xlist2); ytest = np.array(ylist2); etest = np.array(elist2)

### Visualizing data

How do you know the data was read in correctly? You have seen an image of what the Moho estimates look like in the lecture, on the EARS web site, and perhaps via your own mapping efforts. To make sure you've actually and correctly read in the same data, you should map the data you read in and compare it with what you know the data should look like. 

We can easily visualize (map) the training data points in a Cartesian coordinate system, pretending that a degree of latitude equals a degree of longitude, for example. The data values (Moho depth) can be plotted as colored symbols, as is done on the EARS site. Also as done on the EARS site, include a colorbar with your plot.


In [ ]:
# Quick & Dirty data check:
# latitudes:
xamp = np.concatenate((xobs[:,1], xtest[:,1]))     # merge training and testing set for the moment
# and longitudes:
xomp = np.concatenate((xobs[:,0], xtest[:,0]))   
# and Moho depths:
Md = np.concatenate((yobs, ytest))
err = np.concatenate((eobs,etest))

plt.scatter(xomp,xamp, s=10., c=Md, cmap='nipy_spectral',vmin=18,vmax=66)  
plt.colorbar()   

Make sure your plots look like you have the same data set as plotted on the EARS web site. I did that by choosing the `nipy_spectral` color map.
Obviously we can do better, using our new **pyGMT** knowledge. Here we go. 

In [ ]:
import pygmt

In [ ]:
fig = pygmt.Figure()

fig.coast(
    region=[-130, -60, 25, 52],
    projection='B-95/37/25/52/20c',
    shorelines=True,
    water='lightblue',
    land='lightgrey',
    frame=True)
##
colormap = '       '        # help me out, what color map to use?
##

pygmt.makecpt(cmap=colormap, series=[18, 66])

fig.plot(x=xomp,y=xamp,style='c0.15c',cmap=True,color=Md)

fig.colorbar(frame='+l"Moho depth (km)"')
fig.show()


This shows an interesting and busy distribution of station locations with Moho depth estimates. Do you recognize Moho depth behavior that correlates with geological, tectonic, or topographic provinces?

Examine the measurement errors (noise) by mapping them. Add code below to also plot the errors, using the same projection and symbol size, and a different color bar.

*Hint: use `series = [0, 20]` and `reverse=True`*

In [ ]:
# enter code here to map the formal error estimates in the Moho depth data from EARS:


















You will have noticed there are many low values. Remarkably, some of the uncertainties are zero, which will be a problem later. Replace the zeroes (and any values < 0.5) by `e = 0.5` in both the training and the testing sets. 



In [ ]:
eobs[eobs<0.5]=[0.5]   #removing zeroes in training set data errors
etest[etest<0.5]=[0.5] #removing zeroes in testing set data errors
err = np.concatenate((eobs,etest))

Map the uncertainties again. But first: It looks like we'll be doing a lot of mapping, so we need a quick function for this particular area and projection. Use this new function `mapM()` to map the uncertainties again.

In [ ]:
# make a gmt Moho map function

def mapM(fig,lons,lats,Mohos,lims=[18, 66]):
    """
    IN: figure instance, (longitudes,latitudes, ad Moho depths)
    optional: Moho depth limits for the color palette, default: [18,66]
    OUT: the figure instance with the Moho points plotted on it, color-coded by Moho depth
    """
    fig.coast(
        region=[-130, -60, 25, 52],
        projection='B-95/37/25/52/20c',
        shorelines=True,
        water='lightblue',
        land='lightgrey',
        frame=True)
    pygmt.makecpt(cmap='wysiwyg', series=lims)
    fig.plot(x=lons,y=lats,style='c0.15c',cmap=True,color=Mohos)
    fig.colorbar(frame='+l"Moho depth (km)"')
    return fig

# complete code here to again map the uncertainties (after removing the zeroes):
# Hints: open a pyGMT figure, call mapM(), show the pyGMT figure.






In [ ]:
# now just plot the training data on a map

f = pygmt.Figure()
mapM(f,xobs[:,0],xobs[:,1],yobs)
f.show()

In [ ]:
# now just plot the testing data:

f = pygmt.Figure()
mapM(f,xtest[:,0],xtest[:,1],ytest)
f.show()

## Time to get started

Our goal is to turn the data into a function that predicts the Moho depth at any longitude and latitude, including the coordinates for the Moho values of the test set. 

### Method 1 (no grid)

You learned about kriging in the previous unit of this Seismology Summer School. Kriging is a great way to use actual data points to predict data values at points where you don't have data, and all that without the need to define a physical model or mathematical function for the data. 

Another way in which we can do this, is as follows. Consider a point $k$ for which we would like to know the data value (Moho depth) but we have no station there and no Moho estimate. However, there are station locations in the surrounding region that we do have Moho depth estimates for. We can estimate the Moho depth at point $k$ by simply calculating a weighted average of Moho depth at points in the surrounding region. The weights $f_i (d_{ik},\epsilon_i)$ of the points are determined by how far away a point $i$ is from point $k$ and, if appropriate, by the error estimates $e_i$ in the Moho depth estimates. 

This obviously requires a function of distance between the points on the surface of the Earth. Let's define two different types of basisfunctions, the squared exponential, and the bi-linear interpolation function:


In [ ]:
# Squared exponential (SE) function (a Gaussian-shaped function)
# IN: two function parameters, A (amplitude) and w (halfwidth at halfheight), and
# two arrays of position information, x1 and x2
# each element of x1 and x2 can be a D-dimentional position vector. Here we use only 1-D or 2-D inputs
# OUT: a len(x1) by len(x2) array of SE function values calculated for distances between each point in x1 and each point in x2

def SEfunction(par, x1, x2):
    A, w = par
    w = w*111.18                   # convert from degrees to km
    Gamma = 0.5/w**2
    D = gcdist(x1,x2)   
    return A*np.exp(-Gamma*D**2)   
    

# Bilinear interpolation (BI) function
# IN: two function parameters, A (amplitude) and w (halfwidth at halfheight), and
# two arrays of position information x1, and x2
# each element of x1 and x2 can be a 1- or 2-dimentional position vector.
# OUT: a len(x1) by len(x2) array of BIfunction values calculated for distances between each point in x1 and each point in x2

def BIfunction(par, x1, x2):
    A = par[0]; w = 2*par[1]
    dim = len(np.shape(x1))
    Db = np.zeros([len(x1),len(x2)])
    i = 0
    if dim == 1:
        for x in x2:
            lond = abs(x-x1[:])/w
            lond[lond>1] = 1
            Db[:,i] = (1-lond)
            i += 1
    elif dim == 2:
        for x in x2:
            lond = abs(x[0]-x1[:,0])/w
            latd = abs(x[1]-x1[:,1])/w
            lond[lond>1] = 1; latd[latd>1] = 1
            Db[:,i] = (1-lond)*(1-latd)
            i += 1
    return A*Db


#### Weighted averages (essentially a nearest neighbor approach)

Let's create a baseline Moho model that predicts Moho depth at a location simply by averaging Moho measurements made nearby. These predictions can be made for arbitrarily ditributed locations like our testing points, or for locations on a regular grid.

$$
y(\mathbf{x}_k) = \frac{\sum_{i=0}^{N} f_i (d_{ik},\epsilon_i) \cdot y(\mathbf{x}_i)}{\sum_{i=0}^{N} f_i (d_{ik},\epsilon_i)} ,
$$

where

$$
f_i (d_{ik},\epsilon_i) = \frac{k(\mathbf{x}_k,\mathbf{x}_i)}{\epsilon_i} ,
$$

and $N$ is the number of data points, $y(\mathbf{x}_i)$ is the $i$-th data point, $\mathbf{x}_k$ is the $k$-th point at which wish to know the Moho depth $y(\mathbf{x}_k)$, and $k(\mathbf{x}_k,\mathbf{x}_i)$ is the squared exponential (radial/spherical/Gaussian) function of the distance between the $i-th$ and $k-th$ locations. 

Now calculate $y(\mathbf{x_k})$ for each point in the testing set and plot the results. 

*Hint: Set A=1 and w=0.7* 


In [ ]:
A =           # complete
w =           # complete
par = [A,w]

fall = SEfunction(par,xtest,xobs)/eobs    # (2D array of f_ki)
yfall = yobs*fall                           # (2D array of numerators)
yout = np.sum(yfall,axis=1)/np.sum(fall,axis=1)

# Here is some space for you to compute and report a misfit value between yout and ytest:





f = pygmt.Figure()
mapM(f,xtest[:,0],xtest[:,1],yout)
f.show()


Now set up a grid of points that are equally spaced in latitude and longitude. Then calculate $y(\mathbf{x_k})$ for each point in this grid and plot the results.  


In [ ]:
# enter code here to compute and visualize Moho depth on points on a grid 
# via averaging of Moho depth at nearby stations.














### Method 2 (with grid)

This is were we will be using our knowledge of inverse methods, using theory from the accompanying lecture for unit11. Recall from the lecture we wish to estimate $\mathbf{w}$ by inverting $\mathbf{y} = \mathbf{G} \mathbf{w}$. The following function solves for $\mathbf{w}$, given the data (Moho depths) $\mathbf{y}$ and basisfunction values in $\mathbf{G}$. It also genrates the grid 


In [ ]:

# IN: observations (yobs) at support points (xobs) with errors eobs, and two parameters (Covpar)
# OUT: support points for basisfunctions, weights gw, covariance of gw, and misfit to training data

def Invertw(CovPar, xobs, yobs, eobs, xpars):
    A, w = CovPar
    Cdh = eobs*np.identity(len(yobs)); Cdhi = inv(Cdh)
    Cd = np.dot(Cdh,Cdh.T); Cdi = inv(Cd)
    Cmh = np.sqrt(A)*np.identity(len(xpars)); Cmhi = inv(Cmh)
    Cm = np.dot(Cmh,Cmh.T); Cmi = inv(Cm)                           
    G = BIfunction([1,w], xobs, xpars)  
    Ai = inv(np.dot(G.T,np.dot(Cdi,G)) + Cmi)
    Ggi = np.dot(Ai,np.dot(G.T,Cdi))
    gw = np.dot(Ggi,yobs)
    covw = np.dot(Ggi,np.dot(G,Ai))   
    dmobs = np.dot(G,gw)
    diff = (yobs-dmobs)/eobs
    misfit = np.sqrt(np.dot(diff,diff)/len(diff))
    return gw, covw, misfit
    

In [ ]:
# Let's go and compute gw = Mg (Moho at grid points), then visualize results. 
A = 3          
w = 0.5
par = [A,w]

# first define a regular grid, with support points spaced 2w apart:
xlist = []
latitudes = np.arange(np.min(xobs[:,1]),np.max(xobs[:,1]),2*w)
longitudes = np.arange(np.min(xobs[:,0]),np.max(xobs[:,0]),2*w)
for xa in latitudes:
    for xo in longitudes:
        xlist.append([xo,xa])
xpars = np.array(xlist)

# Inversion estimates deviations centered on a mean -- let's set the spatially averaged Moho depth to 40 km
# and invert for deviations of it:
mm = 40

gw, covw, mf = Invertw(par, xobs, yobs-mm, eobs, xpars)
Mg = mm + gw

f = pygmt.Figure()
mapM(f,xpars[:,0],xpars[:,1],Mg)
f.show()

In [ ]:
# How about using the gw to actually compute Moho depth as Mdg = G * w ?

G = BIfunction([1,w], xpars, xpars)
Mdg = np.dot(G,gw) + mm

f = pygmt.Figure()
mapM(f,xpars[:,0],xpars[:,1],Mdg)
f.show()



**Are these last two maps the same? Are they supposed to be the same? Why are they (not) the same?**


In [ ]:
# Here you can use the inferred gw to compute the Moho depth at the testing points.

G =                   # complete
Mdtest = npdot(G,gw)

# compute and report the misfit between Mdtest and ytest:




# plot Mdtest:






We are now ready to plot this grid as the continuous image that it represents. 

First we'll plot it on its native resolution, with each grid point assuming the value of the Moho depth at the support point and extending it half the grid spacing in all directions. 

Second, we'll take advantage of the continuous function defined by the bi-linear basisfunctions. We can plot this function in any resolution. 

In [ ]:
# import xarray:
import xarray as xr

In [ ]:
# First

layer = np.array([0])
Mohray = np.zeros((len(latitudes),len(longitudes),len(layer)))
k = 0
for i in np.arange(len(latitudes)):
    for j in np.arange(len(longitudes)):
        Mohray[i,j,0] = Mg[k]
        k += 1

MohoGrid1 = xr.DataArray(Mohray, coords=(latitudes, longitudes, layer), dims=('lat', 'lon', 'layer'))
#print(MohoGrid1.coords)

fig = pygmt.Figure()

colormap = 'wysiwyg' 
pygmt.makecpt(cmap=colormap, series=[18, 66])

fig.grdimage(
    MohoGrid1.sel(layer=0),
    region=[-130, -60, 25, 52],
    projection='B-95/37/25/52/20c',
    cmap=colormap,
)

fig.coast(
    region=[-130, -60, 25, 52],
    projection='B-95/37/25/52/20c',
    shorelines=True,
    frame=True)

fig.colorbar(frame='+l"Moho depth (km)"')
fig.show()



In [ ]:
# Second:

# Finer grid, just for visualization. Model resolution remains at 1 degree.
# this is extremly inefficient and slow....
# ... as it involves the calculation of many distances that are subsequently multiplied by zero.
flats = np.arange(np.min(xobs[:,1]),np.max(xobs[:,1]),0.3)
flons = np.arange(np.min(xobs[:,0]),np.max(xobs[:,0]),0.3)  
xlist = []
for xa in flats:
    for xo in flons:
        xlist.append([xo,xa])
xfine = np.array(xlist)

Gf = BIfunction([1,w], xfine, xpars)
Mfine = np.dot(Gf,gw) + mm

Mfgx = np.zeros((len(flats),len(flons),len(layer)))
k = 0
for i in np.arange(len(flats)):
    for j in np.arange(len(flons)):
        Mfgx[i,j,0] = Mfine[k]
        k += 1

MohoGrid2 = xr.DataArray(Mfgx, coords=(flats, flons, layer), dims=('lat', 'lon', 'layer'))
#print(MohoGrid2.coords)

fig = pygmt.Figure()

colormap = 'wysiwyg' 
pygmt.makecpt(cmap=colormap, series=[18, 66])

fig.grdimage(
    MohoGrid2.sel(layer=0),
    region=[-130, -60, 25, 52],
    projection='B-95/37/25/52/20c',
    cmap=colormap,
)

fig.coast(
    region=[-130, -60, 25, 52],
    projection='B-95/37/25/52/20c',
    shorelines=True,
    frame=True)

fig.colorbar(frame='+l"Moho depth (km)"')
fig.show()
    

### Notes:

- The method above that took so long is conceptually simple, but computationally *inefficient*. In reality, one limits the number of basisfunctions to sum over only those that have non-zero values at the point of interest. For bi-linear interpolation functions, one needs to only find the *four basisfunctions* centered on the corners of the grid square that contains the point of interest. 


- *Ma and Lowry* utilized both inverse methods and kriging, as well as additional data, notably gravity data, to map the depth to the Moho beneath the USA in the following paper: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017TC004540. They did a far better job than what we could have done in this problem set.


- **Tomography**: There is little time left to cover tomography. In words: We have written the value of some Earth property (e.g. Moho depth, or seismic velocity) at a particular point (defined by its latitude and longitude) as a sum of weights times local basis functions.  In tomography, data are not point measurements but sum or integral measurements of properties experienced by a wave along a path (or within a sensitivity kernel). That path can be sampled finely, i.e. defined by a long string (or volume) of points. The measured data can be expressed as a weighted, perhaps transformed (e.g. from velocity to time traveled) sum over the properties at all the points on that path (or within a sensitiviy kernel). The property at one (and all) of points on the path can be expressed in terms of a sum over local basisfunctions. We considered basisfunctions over a two-dimensional surface (a map) but they can be defined over a three-dimensional volume as well, which is useful when adding a depth dimension for a tomography model. *Combining the two integrals* (weighted sum along ray path & weighted sum over basis functions) provides the rows for matrix $\mathbf{G}$ in $\mathbf{d} = \mathbf{Gm}$, where $\mathbf{d}$ is the measured data (e.g. arrival time delays), and $\mathbf{m}$ represents the Earth properties you are inverting for. 


- If you ever want to explain the benefits and pitfalls of seismic tomography to non-seismology colleagues feel free to use a lecture I once prepared for geologists at an EarthScope workshop: http://geophysics.earth.northwestern.edu/seismology/suzan/ES2007_minicourse.pdf


## Break-out Room Work


1. Plot error estimates for Moho depths contained in the EARS data set before *and* after increasing ~0 errors to 0.5 km.


2. This notebook used the nearest neighbor method to predict Moho depths at TA station locations. You should add a cell or cells that use the same method to predict (and plot) Moho depths at points on a regular grid. 


3. In this notebook we frequently used non-TA data to predict Moho depth at TA station locations, where we also have measurements. This provides you with an excellent opportunity to test your modeling. For each set of predictions, compare them with the actually reported Moho depths for TA stations (`ytest`) by computing and reporting and/or visualizing differences/misfits between the EARS Moho depths and the ones you predicted. 


4. The variable w represents the width of the bi-linear and/or squared-exponential basis functions. Vary w within reasonable bounds and understand its effect on the results. Keep reporting/visualizing misfits for various values of w. 


5. Do better. We only used the non-TA data for gridding and predicting. You can use *all* EARS data together to construct a better grid of the Moho of 48 states of the USA. 


6. Keep discussing this with your peers, ask questions of each and other and your instructors, provide feedback and recommendations for the entire summer school.


7. Nominate a black seismologist from within or outside of ROSES for the AGU Seismology Section Social Media Spotlight.


8. Celebrate that you are not receiving a grade for this course. This was all about learning useful stuff for research. Cheers!



## Challenge 

A final course-wide challenge may or may not be posted on Slack in one week. Stay tuned and keep checking Slack. You are always welcome to pick up previously posted challenges and post results at any time! *Hank Cole* has volunteered to preserve course materials, *Liam Toney, Sydney Dybing, and Elizabeth Berg* have provide fabulous and expert TA support, and the entire ROSES summer school would not have happened without the tireless and expert efforts of **Fransiska Dannemann**.

# THANK YOU
